We are uploading all csv files of the Scarlet & Violet main and special sets.

Sets: SV01, SV02, SV03, SV: Scarlet & Violet 151, SV04, SV: Paldean Fates, SV05, SV06, SV: Shrouded Fable, SV07, SV08, SV: Prismatic Evolutions, SV09

In [10]:
import pandas as pd
import scipy as sp

sv01_df = pd.read_csv('SV01ScarletAndVioletBaseSetProductsAndPrices.csv')
sv02_df = pd.read_csv('SV02PaldeaEvolvedProductsAndPrices.csv')
sv03_df = pd.read_csv('SV03ObsidianFlamesProductsAndPrices.csv')
sv04_df = pd.read_csv('SV04ParadoxRiftProductsAndPrices.csv')
sv05_df = pd.read_csv('SV05TemporalForcesProductsAndPrices.csv')
sv06_df = pd.read_csv('SV06TwilightMasqueradeProductsAndPrices.csv')
sv07_df = pd.read_csv('SV07StellarCrownProductsAndPrices.csv')
sv08_df = pd.read_csv('SV08SurgingSparksProductsAndPrices.csv')
sv09_df = pd.read_csv('SV09JourneyTogetherProductsAndPrices.csv')
sv_151_df = pd.read_csv('SVScarletAndViolet151ProductsAndPrices.csv')
sv_pf_df = pd.read_csv('SVPaldeanFatesProductsAndPrices.csv')
sv_sf_df = pd.read_csv('SVShroudedFableProductsAndPrices.csv')
sv_pe_df = pd.read_csv('SVPrismaticEvolutionsProductsAndPrices.csv')

We then merge all of these sets into a master set, filter out unnecessary columns, and add a column for the textual set name based on groupId variable.

In [11]:
master_df = pd.concat([sv01_df, sv02_df, sv03_df, sv04_df, sv05_df, sv06_df, sv07_df, sv08_df, sv09_df, sv_151_df, sv_pf_df, sv_sf_df, sv_pe_df])
master_df = master_df[['productId', 'name', 'groupId', 'lowPrice', 'highPrice', 'marketPrice', 'extRarity']]

# filter for cards of specific rarity
master_df = master_df[(master_df['extRarity'] == 'Special Illustration Rare') | (master_df['extRarity'] == 'Illustration Rare') | (master_df['extRarity'] == 'Hyper Rare')]

# add textual set name column
def get_textset(setId):
    setId_list = [22873, 23120, 23228, 23286, 23381, 23473, 23537, 23651, 24073, 23237, 23353, 23529, 23821]
    textId_list = [
        'Scarlet And Violet Base',
        'Paldea Evolved',
        'Obsidian Flames',
        'Paradox Rift',
        'Temporal Forces',
        'Twilight Masquerade',
        'Stellar Crown',
        'Surging Sparks',
        'Journey Together',
        'Scarlet And Violet 151',
        'Paldean Fates',
        'Shrouded Fable',
        'Prismatic Evolutions'
    ]
    index = setId_list.index(setId)
    return textId_list[index]

master_df['set'] = master_df['groupId'].apply(get_textset)

# add release date
def get_dateset(setId):
    name_list = ['Scarlet And Violet Base',
        'Paldea Evolved',
        'Obsidian Flames',
        'Paradox Rift',
        'Temporal Forces',
        'Twilight Masquerade',
        'Stellar Crown',
        'Surging Sparks',
        'Journey Together',
        'Scarlet And Violet 151',
        'Paldean Fates',
        'Shrouded Fable',
        'Prismatic Evolutions']
    date_list = [
        '05-31-2023',
        '06-09-2023',
        '08-11-2023',
        '10-03-2023',
        '03-22-2024',
        '05-24-2024',
        '09-13-2024',
        '11-08-2024',
        '03-28-2025',
        '09-22-2023',
        '01-26-2024',
        '09-02-2024',
        '01-17-2025'
    ]
    index = name_list.index(setId)
    return date_list[index]

master_df['set'] = master_df['groupId'].apply(get_textset)
master_df['release date'] = master_df['set'].apply(get_dateset)

# sort dataframe by card rarity
master_df = master_df.sort_values(by=['groupId', 'extRarity'])

# reset index
master_df = master_df.reset_index()
master_df.index +=1

master_df['release date'] = pd.to_datetime(master_df['release date'], format='%m-%d-%Y')

display(master_df)

,index,productId,name,groupId,lowPrice,highPrice,marketPrice,extRarity,set,release date
1,437,490043,Miraidon ex - 253/198,22873,4.99,60.00,5.76,Hyper Rare,Scarlet And Violet Base,2023-05-31
2,439,490045,Koraidon ex - 254/198,22873,3.00,28.20,5.50,Hyper Rare,Scarlet And Violet Base,2023-05-31
3,483,490294,Basic Fighting Energy - 258/198,22873,2.39,25.00,3.14,Hyper Rare,Scarlet And Violet Base,2023-05-31
4,484,490295,Basic Lightning Energy - 257/198,22873,3.00,50.00,5.01,Hyper Rare,Scarlet And Violet Base,2023-05-31
5,485,490296,Rare Candy - 256/198,22873,5.80,30.02,7.83,Hyper Rare,Scarlet And Violet Base,2023-05-31
...,...,...,...,...,...,...,...,...,...,...
437,338,623610,Iono's Bellibolt ex - 183/159,24073,135.00,180.00,136.62,Special Illustration Rare,Journey Together,2025-03-28
438,339,623611,Lillie's Clefairy ex - 184/159,24073,300.00,329.36,328.63,Special Illustration Rare,Journey Together,2025-03-28
439,340,623612,N's Zoroark ex - 185/159,24073,214.10,219.99,219.62,Special Illustration Rare,Journey Together,2025-03-28
440,341,623613,Hop's Zacian ex - 186/159,24073,175.00,179.99,178.79,Special Illustration Rare,Journey Together,2025-03-28


We will also have to fill in all NaN values, especially for the market prices of recent sets (ie. SV09 Journey Together).

In order to fill in the NaN values, we will take the average of the lowPrice and highPrice values and input the average into marketPrice.

In [12]:
def fill_market_price(row):
    if pd.isna(row['marketPrice']):
        low_high_average = (row['lowPrice'] + row['highPrice'])/2
        row['marketPrice'] = round(low_high_average, 2)
    return row['marketPrice']

master_df['marketPrice'] = master_df.apply(fill_market_price, axis=1)

We will now conduct three hypothesis tests to observe trends and patterns in data.

TEST I: Using an ANOVA test, we plan on determining if the average price of cards of the same rarity vary significantly across all relevant sets
(relevant sets - scarlet and violet main and special sets)

Because we are testing multiple categories and observing if they exist within the same distribution, an ANOVA test would be appropriate.
Significance level = 0.05 (Level of confidence = 95%)

H0: There is no significant difference in the prices of the same card rarity across the relevant sets.
HA: There exists significant differences in the prices of the same card rarity across the relevant sets.

In [13]:
hr_df = master_df[master_df['extRarity'] == 'Hyper Rare']
ir_df = master_df[master_df['extRarity'] == 'Illustration Rare']
sir_df = master_df[master_df['extRarity'] == 'Special Illustration Rare']

average_prices = pd.DataFrame(columns=['set', 'averagePrice'])

Hyper Rare Cards:

In [14]:
# for Hyper Rare cards
hr_sets = hr_df.groupby('set', sort=False)

for set,group in hr_sets:
    group = group.dropna(subset=['marketPrice']).sort_values(by='marketPrice')
    rarity_mean = group['marketPrice'].mean()
    row = {'set': set, 'averagePrice': round(rarity_mean, 2)}
    average_prices = pd.concat([average_prices, pd.DataFrame([row])], ignore_index=True)

hr_avg_prices = [group['marketPrice'].dropna().values for _, group in hr_df.groupby('set', sort=False)]

hr_res = sp.stats.f_oneway(*hr_avg_prices)
hr_tukey = sp.stats.tukey_hsd(*hr_avg_prices)
print(hr_res.pvalue)
print(hr_tukey)

/var/folders/jf/0fx_tc1j7q53x09tmy3ll_f80000gn/T/ipykernel_32084/3381310859.py:8: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



1.2307944715509516e-06
Tukey's HSD Pairwise Group Comparisons (95.0% Confidence Interval)
Comparison  Statistic  p-value  Lower CI  Upper CI
 (0 - 1)      0.690     1.000   -31.715    33.095
 (0 - 2)     -3.223     1.000   -46.699    40.252
 (0 - 3)     -0.230     1.000   -43.705    43.245
 (0 - 4)     -0.245     1.000   -34.451    33.962
 (0 - 5)     -2.320     1.000   -37.818    33.178
 (0 - 6)     -6.393     1.000   -41.891    29.104
 (0 - 7)     -5.483     1.000   -40.981    30.014
 (0 - 8)    -25.485     0.479   -62.715    11.745
 (0 - 9)     -7.690     1.000   -51.165    35.785
 (0 - 10)    -20.675     0.718   -56.173    14.823
 (0 - 11)    -21.691     0.717   -58.921    15.539
 (0 - 12)    -85.747     0.000  -129.222   -42.271
 (1 - 0)     -0.690     1.000   -33.095    31.715
 (1 - 2)     -3.913     1.000   -44.902    37.076
 (1 - 3)     -0.920     1.000   -41.909    40.069
 (1 - 4)     -0.935     1.000   -31.920    30.050
 (1 - 5)     -3.010     1.000   -35.415    29.395
 (1 - 

TEST III: Pairwise T-Test: We will use pairwise T-tests to determine if there exists a significant difference between the volatility in prices between rarities. Here, we define volatility as the difference between a cards high and low price points. We'll have a significance level of $\alpha = 0.5$. 

$H_0:$ There is no significant difference in the volatility in prices between rarities. 

$H_a:$ There is significant difference in the volatility in prices between rarities. 

In [15]:
from scipy.stats import ttest_ind

master_df['volatility'] = master_df['highPrice'].astype(float) - master_df['lowPrice'].astype(float)

ttest_hr_df = master_df[master_df['extRarity'] == 'Hyper Rare']
ttest_ir_df = master_df[master_df['extRarity'] == 'Illustration Rare']
ttest_sir_df = master_df[master_df['extRarity'] == 'Special Illustration Rare']

hr_vs_ir = ttest_ind(ttest_hr_df['volatility'], ttest_ir_df['volatility'])
hr_vs_sir = ttest_ind(ttest_hr_df['volatility'], ttest_sir_df['volatility'])
ir_vs_sir = ttest_ind(ttest_ir_df['volatility'], ttest_sir_df['volatility'])

print(f"The observed p-value of hyper rare cards versus illustration rare is: {hr_vs_ir[1]}")
print(f"The observed p-value of hyper rare cards versus special illustration rare is: {hr_vs_sir[1]}")
print(f"The observed p-value of illustration rare cards versus special illustration rare is: {ir_vs_sir[1]}")

The observed p-value of hyper rare cards versus illustration rare is: 0.32509098167592854
The observed p-value of hyper rare cards versus special illustration rare is: 0.08174837546624164
The observed p-value of illustration rare cards versus special illustration rare is: 0.5146676677454878


Since the observed p-values of all three categories are greater than our significant level of $\alpha = 0.5$ we fail to reject our null hypothesis. There is not strong evidence that suggests significant differences in volatility in prices between rarities.

In [16]:
import plotly.express as plotly
import plotly.io as pio

fig_ttest = plotly.violin(
    master_df, 
    y = 'volatility', 
    x = 'extRarity', 
    color ='extRarity',
    box = True, 
    points='all',  \
    title = 'Volatility grouped by Rarity',
    labels={'extRarity': 'ExtRarity', 'volatility': 'Volatility'}
)
fig_ttest